# Overview

**Objectives:**

1.   Fix issue --repetition of same word sequences-- found in text generated by our finetuned version of gpt2-spanish (See Finetuning_gpt2.ipynb)
2.   Generate open-ended text in the writing style of Gustavo Adolfo Becquer.

**Input:** Prompt

**Model:** `gpt2-spanish_Becquer-joined.pt` (finetuned)

**Expected Output:** Text generated by the finetuned model in the writing style of our author.


This notebook experiments with the following decoding methods:

*   Greedy search
*   Beam search
*   Sampling



References:

*   [How to generate text: using different decoding methods for language generation with Transformers](https://huggingface.co/blog/how-to-generate)

Follow the code below or [go straight to the results](#conclusion)

# Imports

In [ ]:
!pip install transformers datasets torch
!pip install transformers[torch]
!pip install accelerate -U

In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [3]:
# # Uncomment this cell if the finetuned model was saved in gdrive.

# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish")

# Load model and tokenizer

In [5]:
import tensorflow as tf

In [6]:
model_save_name = 'gpt2-spanish_Becquer-joined.pt'

path = F"/content/gdrive/My Drive/Insert your folders here/{model_save_name}"

model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [7]:
tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")

# Text generation

## Greedy search

In [8]:
prompt = "Será verdad que"  # Change this to your desired starting prompt
max_length = 150  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

output = model.generate(input_ids, max_length=max_length, temperature=0.7)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(output_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Será verdad que no hay nada que decir.

--¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh!


To get rid of `Warning The attention mask and the pad token id were not set.`, set `pad_token_id`.

In [9]:
prompt = "Será verdad que"  # Change this to your desired starting prompt
max_length = 150  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

output = model.generate(input_ids, max_length=max_length, temperature=0.7, pad_token_id=50256)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(output_text)


Será verdad que no hay nada que decir.

--¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh!


Changing temperature yields the same result.

In [10]:
prompt = "Será verdad que"  # Change this to your desired starting prompt
max_length = 75  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

output = model.generate(input_ids, max_length=max_length, temperature=0.1, pad_token_id=50256)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(output_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Será verdad que no hay nada que decir.

--¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh! ¡Oh!


## Beam search

Based on [this article](https://huggingface.co/blog/how-to-generate), repetition can be avoided with Beam search.

In [11]:
prompt = "Será verdad que"  # Change this to your desired starting prompt
max_length = 175  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,
    max_new_tokens=50,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Será verdad que no hay nada que decir.

  

  

  

  


































Changing the prompt and number of beams.

In [12]:
prompt = "Mientras flotaban en"  # Change this to your desired starting prompt
# max_length = 75  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,
    max_new_tokens=50,
    num_beams=7,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Mientras flotaban en el aire las hojas de los árboles.

  

  

  

  

  

  

  

  

  

  

  

  


  




Introduce n-gram penalties

In [13]:
prompt = "Mientras flotaban en"  # Change this to your desired starting prompt
# max_length = 75  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,
    max_new_tokens=150,
    num_beams=7,
    no_repeat_ngram_size=2,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Mientras flotaban en el aire las hojas de los árboles.

  

--¡Oh!--exclamó el joven, que se había puesto en pie de un salto y se disponía á salir de la
puerta, cuando de improviso se oyó una voz que decía:--¿Quién es usted? ¿Qué quiere usted que le diga?
El joven se detuvo un momento, y después, volviéndose hacia el
caballero, le dijo con voz temblorosa y temblorosa.--Señor conde, ¿es usted el señor conde de Monteclaro? ¡Ah! ¡Dios mío! ¿Es usted, acaso, el mismo que me ha
adivinado en su lecho de muerte? Pues bien, yo soy el único que puede
haber visto en


The above technique starts to generate text resembling our author! This is an example:

`Mientras flotaban en el aire las hojas de los árboles.
--¡Oh!--exclamó el joven, que se había puesto en pie de un salto y se disponía á salir de la
puerta, cuando de improviso se oyó una voz que decía:--¿Quién es usted? ¿Qué quiere usted que le diga?
El joven se detuvo un momento, y después, volviéndose hacia el
caballero, le dijo con voz temblorosa y temblorosa.--Señor conde, ¿es usted el señor conde de Monteclaro? ¡Ah! ¡Dios mío! ¿Es usted, acaso, el mismo que me ha
adivinado en su lecho de muerte? Pues bien, yo soy el único que puede
haber visto en`

Compare the top beams after generation and choose the one that fits our purpose best.
  * Added `do_sample=True` as suggested [here](https://github.com/huggingface/transformers/issues/2415), because I was getting exactly the same output for each of the 5 sequences generated.

In [14]:
prompt = "Mientras flotaban en"  # Change this to your desired starting prompt
# max_length = 75  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids,
    max_new_tokens=70,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    do_sample=True,
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: Mientras flotaban en el aire,
depositaron sus cabezas sobre la tierra, y la luz de la luna se derramó
en el cielo, iluminando la noche.

Al cabo de un rato, cuando el sol se había puesto, comenzó á
suturar las nubes de polvo, que se arremolinaban en torno á la hoguera
que ardía en
1: Mientras flotaban en el aire,
depositaron sus cabezas sobre la tierra, y la luz de la luna se derramó
en el cielo, iluminando la noche.

Al cabo de un rato, cuando el sol se había puesto, comenzó á
suturar las nubes de polvo, que se arremolinaban en torno de las
palabras de los
2: Mientras flotaban en el aire,
depositaron sus cabezas sobre la tierra, y la luz de la luna se derramó
en el cielo, iluminando la noche.

Al cabo de un rato, cuando el sol se había puesto, comenzó á
suturar las nubes de polvo, que se arremolinaban en torno á la hoguera
que ardía sobre
3: Mientras flotaban en el aire,
dep

Experiment with less beams and more ngrams:

In [15]:
prompt = "Mientras flotaban en"  # Change this to your desired starting prompt
# max_length = 75  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids,
    max_new_tokens=70,
    num_beams=3,
    no_repeat_ngram_size=4,
    num_return_sequences=3,
    do_sample=True,
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: Mientras flotaban en el aire las nubes de polvo.

  

     


   *    * *


  *  *

--¡Hola!

--Hola. ¿Qué tal?

--Bien. ¿Y tú? 

--Bien, bien. ¿Cómo estás? 

1: Mientras flotaban en el aire las nubes de polvo.

  

     


   *    * *


  *  *

--¡Hola!

--Hola. ¿Qué tal?

--Bien. ¿Y tú? 

--Bien, bien. ¿Cómo estás?

2: Mientras flotaban en el aire las nubes de polvo.

  

     


   *    * *


  *  *

--¡Hola!

--Hola. ¿Qué tal?

--Bien. ¿Y tú? 

--Bien, bien. ¿Y qué tal?


## Sampling

Beam Search can work very well in cases where the max length is more or less predictable, as in Machine Translation or Summarization, but not so much in open-ended generation. Trying sampling as shown [here](https://huggingface.co/blog/how-to-generate#:~:text=introduce%20some%20randomness%20%F0%9F%A4%AA.-,Sampling,-In%20its%20most)

In [16]:
# # set seed to reproduce results. Feel free to change the seed though to get different results
from transformers import set_seed
set_seed(42)

prompt = "Mientras flotaban en"  # Change this to your desired starting prompt
# max_length = 75  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids,
    max_new_tokens=70,
    do_sample=True,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Mientras flotaban en el aire la luz de la luna.

La brisa comenzaba á refrescar el silencio; los árboles se ponían yerbustos y marchitos. La vegetación, densa y menuda, se destacaba.

Pero todo era silencio. La rama que había colocado bajo la rama inundaba con su tinte el espacio que existía entre las
labores del recuerdo


Cooling down the temperature to, hopefully, obtain less gibberish.

In [17]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

prompt = "Mientras flotaban en"  # Change this to your desired starting prompt
# max_length = 75  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids,
    max_new_tokens=70,
    do_sample=True,
    top_k=0,
    temperature=0.6,
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Mientras flotaban en el aire,
de la noche, sobre las rocas de la orilla, se oyó un ruido extraño y espantoso.


--¿Quién vive?--exclamó el joven, muy pálido,
pues de un salto se arrojó sobre el fugitivo, que se había
inclinado sobre su caballo y le dió una terrible sacudida
y el caballo cayó


This is starting to look promising...

Change temp. slightly to see how much it varies--semantic mistakes show up promptly by doing this:

In [18]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

prompt = "Mientras flotaban en"  # Change this to your desired starting prompt
# max_length = 75  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids,
    max_new_tokens=70,
    do_sample=True,
    top_k=0,
    temperature=0.7,
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Mientras flotaban en el aire la luz de la luna.


Hábilmente, el pastor se sentó junto al lecho y después de permanecer un momento inmóvil, como para dar el último
radio.

Pero el pastor, que había estado escuchando como un espectador, no acertaba á contestar;
no obstante, cuando volvió á hablar, el


## Top-K sampling

In [Top-K sampling](https://huggingface.co/blog/how-to-generate#:~:text=problems%20as%20before.-,Top%2DK%20Sampling,-Fan%20et.%20al), the K most likely next words are filtered and the probability mass is redistributed among only those K next words. GPT2 adopted this sampling scheme, which was one of the reasons for its success in story generation.

This still sounds like gibberish, but it does resemble the following stylistic attributes present in the sample books:

*   Use of vivid and descriptive language to help create mental images.
*   Old Spanish typography, like the accent mark in the preposition "á".
*   Longer sentences with detailed descriptions.
*   Formal register present in adverbial and verbal expressions.

In [19]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

prompt = "Mientras flotaban en"  # Change this to your desired starting prompt
# max_length = 75  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# set top_k to 50
sample_output = model.generate(
    input_ids,
    max_new_tokens=70,
    do_sample=True,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Mientras flotaban en el aire la luz de la luna.

La brisa comenzaba á refrescar el silencio; los árboles se ponían en movimiento.

Los pájaros cantaban, á la luz de las lámparas, el viento de un
recuerdo sobre las hojas que se movían bajo sus pies. Entre tanto, el viento se había hecho más
poco en el


## Top-p (nucleus) sampling

Instead of sampling only from the most likely K words, in Top-p sampling chooses from the smallest possible set of words whose cumulative probability exceeds the probability p. See visual [here](https://huggingface.co/blog/how-to-generate#:~:text=Top%2Dp%20(nucleus)%20sampling)

In [20]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

prompt = "Vivo en Berlín"  # Change this to your desired starting prompt
# max_length = 75  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# set top_k to 50
sample_output = model.generate(
    input_ids,
    max_new_tokens=70,
    do_sample=True,
    top_p=0.92,
    top_k=0,
    temperature=0.9 # Added temperature to see if we got a response based on facts from the document.
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Vivo en Berlín; donde la Orden de San Fernando era un medio para conocer la verdad, la incertidumbre y la curiosidad;
en el desierto de Madrid, á orillas del Neva, el bullicio de la multitud.

Pero todo era distinto: el pequeño monasterio de La Caya, el monasterio del Alminar,
el monasterio que más había temido,


* Top-p can also be used in combination with Top-K, which can avoid very low ranked words while allowing for some dynamic selection.

* To get multiple independently sampled outputs, we can again set the parameter num_return_sequences > 1:

In [21]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

prompt = "Mientras flotaban en"  # Change this to your desired starting prompt
# max_length = 75  # Change this to desired output length

input_ids = tokenizer.encode(prompt, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    max_new_tokens=70,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3,
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: Mientras flotaban en el aire la luz de la luna.

La brisa comenzaba á refrescar el silencio; los árboles se ponían en movimiento.

Los pájaros cantaban, á la luz de las lámparas, el viento de un
punto lejano, y unos que arrastraban unas hierbas secas y secas sobre sus cabezas; el viento
se revolvía á su alrededor y
1: Mientras flotaban en el aire, y ya no se oía más allá del aire,
todo se veía muy lejano, aunque los pájaros, muy asustados
de las notas de las trompetas, se apiñaban detrás de sus copas de
madera y se movían á la orilla de la laguna, donde las aguas se volvían á
hablar por sobre el agua, con
2: Mientras flotaban en los aire la negra silueta de sus hombros.

El viento le agitó en los cabellos la cabellera, y
cuando lo soltó ya no pudo contener el grito. De este modo, se había salvado del
ruido del aire, pero su vista estaba llena del blanco
suelo del cielo y sus 

<a name="conclusion"></a>
# Conclusion

Resemblance of the writing style of our author is present in the output generated by the finetuned model when using the following decoding methods:

*   Beam search (with n-gram penalties)
*   Top-K and Top-p sampling

Stylistic attributes that were replicated:
*   Use of vivid and descriptive language to help create mental images.
*   Old Spanish typography, like the accent mark in the preposition "á".
*   Longer sentences with detailed descriptions.
*   Formal register present in adverbial and verbal expressions.

Stylistic attributes that were not replicated:
*   Syntactic complexity--our model can create longer sentences, but lacks logical coherence.

## Output example

Model output:

`Mientras flotaban en el aire la luz de la luna. La brisa comenzaba á refrescar el silencio; los árboles se ponían en movimiento. Los pájaros cantaban, á la luz de las lámparas, el viento de un
recuerdo sobre las hojas que se movían bajo sus pies. Entre tanto, el viento se había hecho más
poco en el`

Fragment from original book:

`De todos modos, allá van estas cuartillas, valgan por lo
que valieren: que si alguien de más conocimientos é importancia,
una vez apuntada la idea, la desarrolla y prepara la opinión para
que fructifique, no serán perdidas del todo. Yo, entre tanto, voy á
trazar un tipo bastante original y que desconfío de poder reproducir.
Ya que no de otro modo, y aunque poco valga, contribuiré al éxito de
la predicación con el ejemplo.`